# Opas nettisivujen tutkailuun


Tämän oppaan tarkoitus on tutustuttaa käyttäjä pythonin BeatifulSoup-kirjastoon, jota voidaan hyödyntää hakemaan netistä dataa esim. sääilmoituksia, urheilutuloksia tai mitä tahansa.

Harjoituksessa käydään läpi miten saadaan haettua paikkakuntakohtainen sää ilmatieteenlaitoksen nettisivuilta.  

Tarkastellaan hieman ensiksi paikkakuntien url-osoitteita:

[Kuopio](https://ilmatieteenlaitos.fi/saa/kuopio)


[Siilinjärvi](https://ilmatieteenlaitos.fi/saa/siilinj%C3%A4rvi)


Kun käydään kyseisillä sivuilla, huomataan, että osoite on muotoa:

https://ilmatieteenlaitos.fi/saa/siilinjärvi

https://ilmatieteenlaitos.fi/saa/kuopio

Ts. url koostuu rungosta ja paikkakunnasta. Tällä päästään alkuun.

In [28]:
#Haetaan tarvittavat kirjastot
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np

#Muodostetaan url-osoite
paikkakunta = "kuopio"
urlRunko = "http://ilmatieteenlaitos.fi/saa/" 
#Haetaan lähituntien sääennuste
lyhyt = "?urlforecast=short"
urlOsoite = urlRunko + paikkakunta + lyhyt

#Tarkistetaan, että sivusta on käytössä
sivu = requests.get(urlOsoite)
#Jos status_code on 200, niin homma toimii
if( sivu.status_code == requests.codes.ok):
    print("Yhteyden muodostaminen onnistui.")
else:
    print("Yhteyden muodostaminen epäonnistui.")
    
#Huom! Yllä olevalla koodin pätkällä tarkistetaan vain, että url-osoitteen runko on kunnossa,
#paikkakunta voi kuitenkin olla väärä.


Yhteyden muodostaminen onnistui.


In [29]:
#Haetaan sivun lähdekoodi
soup = BeautifulSoup(sivu.text, 'html.parser')
#Tulostetaan lähdekoodi
print(soup)
    

<!DOCTYPE html>
 <html class="ltr no-js" dir="ltr" lang="fi-FI"> <head> <title>Sää Kuopio - Ilmatieteen laitos</title> <meta charset="utf-8"/> <meta content="IE=edge" http-equiv="X-UA-Compatible"/> <meta content="width=device-width, initial-scale=1" name="viewport"/> <meta content="text/html; charset=utf-8" http-equiv="content-type"/> <meta content="Sää Kuopio. Sääennuste Suomeen ja maailmalle. Lisäksi havainnot Suomesta." lang="fi-FI" name="description"/> <meta content="sää kuopio, sää, paikallissää, säätiedot, sääennuste, säähavainnot, sääkartta, paikkakuntakohtainen sää, 5 vuorokauden ennuste, säähavaintohistoria, viikon sää, hpa, hPa, ilmatieteenlaitos,päivän pituus, auringon nousu, auringon lasku, auringon nousuaika, lumensyvyys, lumitilanne, viiden vuorokauden sää, lumitilanne Suomessa, 5 vrk sää, fmi, ilmanpaine, auringon nousu ja laskuajat, ilma, 5 vuorokauden sää, viikon sää, päivän pituus suomessa, tarkka sää, paikka sää" lang="fi-FI" name="keywords"/> <link href="/fmi-www-th

Yllä oleva näyttää hieman sekavalta ja sieltä tulisi poimia tarvittavat tiedot. Sivun lähdekoodia kannattaa tarkastella lähemmin nettisivulla käyttämällä Inspect element-toimintoa. Toiminnon saa näkyviin hiiren oikealla näpillä avautuvasta valikosta - ainkin Firefox-selaimessa.

In [36]:
#Koska halutaan lukea lähituntien lämpötiloja, on etsittävä minkä luokan alapuolella ne ovat.
#Viemällä hiiren lämpötilan kohdalle ja sen jälkeen inspect element, niin saa näkyviin minkä luokan 
#alapuolella ne ovat. (Itselläni ei ole oikeastaan yhtään kokemusta nettisivujen tekemisestä, niin saatan 
#käyttää täysin väärää termistöä.) 

#Tarkastelemalla sivun lähdekoodia huomataan, että lämpötilat ovat luokan "meteogram-temperatures"-alapuoolla
#tr tarkoittaa taulukkoa hmlt-kielessä
lampötilatLista = soup.find('tr', attrs={'class':'meteogram-temperatures'})
print(lampötilatLista)

<tr class="meteogram-temperatures"> <td> <div class="temperature positive" title="lämpötila 8 °C">8°</div> </td> <td> <div class="temperature positive" title="lämpötila 8 °C">8°</div> </td> <td> <div class="temperature positive" title="lämpötila 7 °C">7°</div> </td> <td> <div class="temperature positive" title="lämpötila 6 °C">6°</div> </td> <td> <div class="temperature positive" title="lämpötila 6 °C">6°</div> </td> <td> <div class="temperature positive" title="lämpötila 6 °C">6°</div> </td> <td> <div class="temperature positive" title="lämpötila 5 °C">5°</div> </td> <td> <div class="temperature positive" title="lämpötila 4 °C">4°</div> </td> <td> <div class="temperature positive" title="lämpötila 4 °C">4°</div> </td> <td> <div class="temperature positive" title="lämpötila 3 °C">3°</div> </td> <td> <div class="temperature positive" title="lämpötila 3 °C">3°</div> </td> <td> <div class="temperature positive" title="lämpötila 3 °C">3°</div> </td> <td> <div class="temperature positive" t

In [37]:
#Vieläkin aika sotkuisen näköistä, kuitenkin seuraavalla komennolla saadaan poimittua 
#pelkkä teksti yllä olevasta lähdekoodista
lähiTuntienLämpötilat = lampötilatLista.text
print(lähiTuntienLämpötilat)

  8°   8°   7°   6°   6°   6°   5°   4°   4°   3°   3°   3°   3°   3°   3°   3°   4°   5°  


In [43]:
#Seuraavaksi luetaan lämpötilat taulukkoon
#Alustetaan taulukko
lämpötilaTaulukko = []
skip = False

#Seuraava koodinpätkä on hyvä esimerkki miten asiat voi tehdä turhan vaikeasti....
for x in range(0,len(lähiTuntienLämpötilat)):
    if(skip):
        x=x+1
    #Suodatetaan välit    
    if(lähiTuntienLämpötilat[x] == " "):
        skip=False
        continue
    #Tarkastetaan onko numero    
    elif(lähiTuntienLämpötilat[x].isdigit() == True):
        #Tarkastetaan onko kaksinumeroinen luku, oletus on että lämpötilat
        #ovat -100 ja +100 välillä
        if(lähiTuntienLämpötilat[x + 1].isdigit() == True):
            #Tarkastetaan onko negatiivinen luku
            if(lähiTuntienLämpötilat[x - 1] == "-"):
                #Lisätään miinus-merkki
                doubleValue = "-" + str(lähiTuntienLämpötilat[x]) + str(lähiTuntienLämpötilat[x+1])
            else:
                doubleValue = str(lähiTuntienLämpötilat[x]) + str(lähiTuntienLämpötilat[x+1])
            #Lisätään taulukkoon    
            lämpötilaTaulukko.append(doubleValue)
            doubleValue = "None"
            skip=True
        #Jos ei ole kaksinumeroinen, on yksinumeroinen    
        else:
            if(lähiTuntienLämpötilat[x-1] == "-"):
                singleValue = "-" + lähiTuntienLämpötilat[x]
            else:    
                singleValue = weather_temp[x]
            lämpötilaTaulukko.append(singleValue)
            skip=False

#Tarkistetaan taulukko            
print(lämpötilaTaulukko)

['8', '8', '7', '6', '6', '6', '5', '4', '4', '3', '3', '3', '3', '3', '3', '3', '4', '5']
